# Downloading aerial imagery from Google

Now that we've got one pair of coordinates for each potential church, we need to get a photo of each one. We use Google's Static Maps API, which comes with free monthly credit for ~100,000 lookups per month: more than enough! We'll download the photos into a separate folder to keep things tidy.

In [1]:
import pandas as pd
import requests
import re
import os

Now to load our API key. The ```enable_API_calls``` variable is there to stop me burning through credit if I make any mistakes...

In [2]:
with open('secrets/key.txt', 'r') as f:
    api_key = f.read()

In [3]:
enable_API_calls = False #change when you want to run a call

Load our data and do some string manipulations to create a nice filename for the images:

In [4]:
places_of_worship = pd.read_csv('places_of_worship.csv')
places_of_worship['filename'] = places_of_worship['tags.name'].apply(lambda x: re.sub('\W','_', x.lower()).replace('__','_'))
places_of_worship.head()

,id,tags.name,tags.denomination,tags.building,lat,lon,coords_imputed,filename
0,144164756,"St Margaret and St Catherine, Aldham",anglican,NaN,51.898162,0.785737,True,st_margaret_and_st_catherine_aldham
1,145482410,"St Michael and All Angels, Copford",anglican,NaN,51.869317,0.809385,True,st_michael_and_all_angels_copford
2,35139156,St. Albright's Church,anglican,NaN,51.883674,0.817888,True,st_albright_s_church
3,146823677,"All Saints, Eight Ash Green",anglican,NaN,51.896594,0.820112,True,all_saints_eight_ash_green
4,913980214,Eight Ash Green Methodist Church,NaN,NaN,51.895918,0.822322,False,eight_ash_green_methodist_church


We define a function to grab the image from the Google API. Documentation for the API is available at __https://developers.google.com/maps/documentation/maps-static/start__. 20 is a very high zoom level, and we want satellite rather than map imagery to get as much detail as possible.

In [5]:
def get_image(lat, lon, path, filename, key, zoom=20, size='640x640', maptype='satellite'):
    request_url = "https://maps.googleapis.com/maps/api/staticmap"
    payload = {'center': ','.join([str(coord) for coord in [lat, lon]]),
               'zoom': zoom,
               'size': size,
               'maptype': maptype,
               'key': key
              }
    
    r = requests.get(request_url, params=payload)
    if r.status_code != 200:
        print('[ERROR]: non-200 status code received to: {}'.format(r.url))
    else:
        with open(os.path.join(path, filename+'.png'), 'wb') as f:
            f.write(r.content)

Finally, we go through the dataframe and grab an image for each entry, then save the updated places of worship dataframe so it includes the filenames:

In [6]:
for place in places_of_worship.itertuples(index=False):
    if enable_API_calls == True:
        get_image(lat=place.lat, lon=place.lon, path='images', filename=place.filename, key=api_key)
        
places_of_worship.to_csv('places_of_worship.csv', index=False)

The results look like this one of St Andrew's in Fingringhoe:

<img src='writeup_images/st_andrew_fingringhoe.png' />

***